In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt


In [4]:
def extract_answer(text):
    pattern = r"(True|False|Don't know)"
    matches = re.findall(pattern, text)
    return matches


In [20]:
# RESPONSE_PATH = "../../../data/analysis_results/Llama_2_13b_chat_hf_prompt_based_binary_response.csv"

RESPONSE_PATH = "../../../data/analysis_results/Llama_2_13b_chat_hf_node_retrieval_rag_based_binary_response.csv"


In [21]:
response_df = pd.read_csv(RESPONSE_PATH)

response_df.loc[:, 'extracted_answer'] = response_df['llm_answer'].apply(extract_answer)


In [22]:
response_df.loc[:, "answer_count"] = response_df.extracted_answer.apply(lambda x:len(x))

response_df_multiple_answers = response_df[response_df.answer_count > 1]
response_df_single_answer = response_df.drop(response_df_multiple_answers.index)
response_df_single_answer.drop("answer_count", axis=1, inplace=True)


response_df_multiple_answers_ = []
for index, row in response_df_multiple_answers.iterrows():
    if row["extracted_answer"][0] == row["extracted_answer"][1]:
        response_df_multiple_answers_.append((row["question"], row["label"], row["llm_answer"], row["extracted_answer"][0]))
    else:
        response_df_multiple_answers_.append((row["question"], row["label"], row["llm_answer"], None))

response_df_multiple_answers_ = pd.DataFrame(response_df_multiple_answers_, columns=["question", "label", "llm_answer", "extracted_answer"])

response_df_final = pd.concat([response_df_single_answer, response_df_multiple_answers_], ignore_index=True)
response_df_final = response_df_final.explode("extracted_answer")



In [23]:
incorrect_answers_because_of_na = response_df_final[response_df_final.extracted_answer.isna()]


In [24]:
response_transform = {
    "True" : True,
    "False" : False
}

response_df_final.extracted_answer = response_df_final.extracted_answer.apply(lambda x:response_transform[x])



In [25]:
correct_response = response_df_final[response_df_final.label == response_df_final.extracted_answer]
incorrect_response = response_df_final[response_df_final.label != response_df_final.extracted_answer]



In [26]:
total_response = response_df_final.shape[0]
correct_response_count = correct_response.shape[0]
incorrect_response_count = incorrect_response.shape[0] + incorrect_answers_because_of_na.shape[0]

In [19]:
correct_response_ = correct_response_count/total_response
incorrect_response_ = incorrect_response_count/total_response


print("Correct response = ",correct_response_)
print("Incorrect response = ",incorrect_response_)



Correct response =  0.8899835796387521
Incorrect response =  0.11001642036124795


In [27]:
correct_response_ = correct_response_count/total_response
incorrect_response_ = incorrect_response_count/total_response


print("Correct response = ",correct_response_)
print("Incorrect response = ",incorrect_response_)



Correct response =  0.8850574712643678
Incorrect response =  0.11494252873563218
